In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader as pdr
import datetime
import numpy as np

In [2]:
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2020,12,31)

In [3]:
# 삼성전자 10년치 일 데이터 - 하나의 일 데이터를 하나의 tick data로 생각
# close price를 p_t 라고 생각
df = pdr.get_data_yahoo('005930.KS', start, end)
tick = df[['Volume', 'Close']]
tick

,Volume,Close
Date,,
2010-01-04,11950800.0,16180.0
2010-01-05,27925850.0,16440.0
2010-01-06,22948850.0,16820.0
2010-01-07,22107950.0,16260.0
2010-01-08,14777550.0,16420.0
...,...,...
2020-12-23,19411326.0,73900.0
2020-12-24,32502870.0,77800.0
2020-12-28,40085044.0,78700.0


In [51]:
def getDailyVol(close, span0=100):
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days = 1))
    df0 = df0[df0>0]
    df0 = pd.Series(close.index[df0 - 1], index = close.index[close.shape[0] - df0.shape[0]:])
    df0 = close.loc[df0.index] / close.loc[df0.values].values - 1
    df0 = df0.ewm(span = span0).std()
    return df0

In [52]:
getDailyVol(tick['Close'])

Date
2010-01-06         NaN
2010-01-07    0.035712
2010-01-08    0.033363
2010-01-11    0.031154
2010-01-12    0.027087
                ...   
2020-12-23    0.021557
2020-12-24    0.023483
2020-12-28    0.023257
2020-12-29    0.023026
2020-12-30    0.022998
Name: Close, Length: 2706, dtype: float64

In [104]:
def applyPtSlOnTl(close, events, ptSl, molecule):
    events_ = events.loc[molecule]
    out = events_[['tl']].copy(deep = True)
    if ptSl[0] > 0 :
        pt = ptSl[0] * events_['trgt']
    else :
        pt = pd.Series(index = events.index)
    if ptSl[1] > 0 :
        sl = -ptSl[1] * events_['trgt']
    else:
        sl = pd.Series(index = events.index)
    for loc, tl in events_['tl'].fillna(close.index[-1]).iteritems():
        df0 = close[loc:tl]
        df0 = (df0 / close[loc] -1) * events_.at[loc,'side']
        out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()
        out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()
    return out

In [145]:
close = tick['Close']
events = pd.DataFrame({'tl' : tick.index + pd.Timedelta(days = 10), 'trgt' : np.repeat(10000, len(tick.index))}, index = tick.index)
ptSl = [2, 1]
molecule = list(tick.index[100:300])

In [106]:
applyPtSlOnTl(close, events, ptSl, molecule)

KeyError: 'side'

In [146]:
events_ = events.loc[molecule]
out = events_[['tl']].copy(deep = True)
if ptSl[0] > 0 :
    pt = ptSl[0] * events_['trgt']
else :
    pt = pd.Series(index = events.index)
if ptSl[1] > 0 :
    sl = -ptSl[1] * events_['trgt']
else:
    sl = pd.Series(index = events.index)
for loc, tl in events_['tl'].fillna(close.index[-1]).iteritems():
    df0 = close[loc:tl]
    df0 = (df0 / close[loc] -1) 
#     out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()
#     out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()
    print(df0)

Date
2010-05-28    0.000000
2010-05-31   -0.002571
2010-06-01   -0.012853
2010-06-03   -0.001285
2010-06-04    0.019280
2010-06-07    0.011568
Name: Close, dtype: float64
Date
2010-05-31    0.000000
2010-06-01   -0.010309
2010-06-03    0.001289
2010-06-04    0.021907
2010-06-07    0.014175
2010-06-08    0.011598
2010-06-09   -0.007732
2010-06-10   -0.003866
Name: Close, dtype: float64
Date
2010-06-01    0.000000
2010-06-03    0.011719
2010-06-04    0.032552
2010-06-07    0.024740
2010-06-08    0.022135
2010-06-09    0.002604
2010-06-10    0.006510
2010-06-11    0.037760
Name: Close, dtype: float64
Date
2010-06-03    0.000000
2010-06-04    0.020592
2010-06-07    0.012870
2010-06-08    0.010296
2010-06-09   -0.009009
2010-06-10   -0.005148
2010-06-11    0.025740
Name: Close, dtype: float64
Date
2010-06-04    0.000000
2010-06-07   -0.007566
2010-06-08   -0.010088
2010-06-09   -0.029004
2010-06-10   -0.025221
2010-06-11    0.005044
2010-06-14    0.017654
Name: Close, dtype: float64
Date
20

,tl,sl,pt
Date,,,
2010-05-28,2010-06-07,NaT,2010-05-28
2010-05-31,2010-06-10,NaT,2010-05-31
2010-06-01,2010-06-11,NaT,2010-06-01
2010-06-03,2010-06-13,NaT,2010-06-03
2010-06-04,2010-06-14,NaT,2010-06-04
...,...,...,...
2011-03-10,2011-03-20,NaT,2011-03-10
2011-03-11,2011-03-21,NaT,2011-03-11
2011-03-14,2011-03-24,NaT,2011-03-14
